In [2]:
import os
import math
import wandb
from sklearn.model_selection import train_test_split
import random
import numpy as np
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import SGD
import tensorflow as tf
import pandas as pd
from collections import OrderedDict
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

def gen_ue_cluster(UE_sort):# UE를 유클리드 distance 기준으로 클러스터링한 리스트를 반환하는 함수
    UE_cluster = []
    UE_group = []
    UE_distance = [] # 2 UE 간의 거리 리스트
    UE_len = len(UE_sort)
    avg_distance = 0
    UE_sort_keys = list(UE_sort.keys())
    for i in range(0, UE_len - 1):
        x1, y1 = i + 1, UE_sort[UE_sort_keys[i]]
        x2, y2 = i+2, UE_sort[UE_sort_keys[i+1]]
        distance = (((x2 - x1) ** 2) + ((y2 - y1) ** 2)) ** 0.5
        avg_distance += distance
        UE_distance.append((UE_sort_keys[i], UE_sort_keys[i+1],distance))
    avg_distance = avg_distance/len(UE_sort)
    
    for i, x in enumerate(UE_distance):
        if UE_distance[-1][0] == x[0]:
            UE_group.append(x[0])
            if x[2] > avg_distance:
                UE_cluster.append(UE_group.copy())
                UE_group.clear()
                UE_group.append(x[1])
                UE_cluster.append(UE_group.copy())
            else:
                UE_group.append(x[1])
                UE_cluster.append(UE_group.copy())
        else:
            if x[2] > avg_distance:
                UE_group.append(x[0])
                UE_cluster.append(UE_group.copy())
                UE_group.clear()
            else:
                UE_group.append(x[0])
            
    
    return UE_cluster
    
def gap(weights): # Global average pooling
    if len(weights.shape) == 4:
        row = np.mean(weights, axis=1)
        result = np.mean(row, axis=0)
    elif len(weights.shape) == 2:
        result = np.mean(weights, axis=0)
    else:
        raise print('가중치 잘못됨')

    return result


def split_ue_group(UE_weights, UE_NUM): # 각 레이어별 가중치 평균보다 높은지 낮은지 나누는 함수
    layers = UE_weights[0].keys()
    concat_weight = {}
    concat_mean_weight = {}

    for layer in layers:
        total = np.zeros(
            (UE_NUM, UE_weights[0][layer][0].shape[-2:][0], UE_weights[0][layer][0].shape[-2:][1]))
        for i, UE in enumerate(UE_weights):
            total[i] = gap(UE[layer][0])
        concat_weight[layer] = total

    for layer in concat_weight.keys():
        concat_mean_weight[layer] = np.mean(concat_weight[layer], axis=0)

    UE_high_low = {}
    for layer in concat_weight.keys():
        true_cnt_list = []
        for x in range(UE_NUM):
            high_low = concat_weight[layer][x] > concat_mean_weight[layer]
            if len(high_low[high_low == True]) >= len(high_low[high_low == False]):
                true_cnt_list.append(True)
            else:
                true_cnt_list.append(False)
        UE_high_low[layer] = true_cnt_list
        
    result = [0 for _ in range(UE_NUM)]
    for layer in UE_high_low.keys():
        for i, x in enumerate(UE_high_low[layer]):
            if x == True:
                result[i] += 1

    d = dict()
    
    for i, x in enumerate(result):
        d[i] = x

    UE_sort = OrderedDict(sorted(d.items(), key=lambda t:t[1]))
    
    return gen_ue_cluster(UE_sort)


def gen_UE_model():
    model = models.Sequential()
    model.add(layers.Conv2D(filters=6, kernel_size=(5, 5),
                            strides=1, activation='sigmoid', input_shape=(32, 32, 1)))
    model.add(layers.AveragePooling2D(pool_size=2, strides=2))
    model.add(layers.Activation('tanh'))
    model.add(layers.Conv2D(filters=16, kernel_size=(
        5, 5), strides=1, activation='tanh'))
    model.add(layers.AveragePooling2D(pool_size=2, strides=2))
    model.add(layers.Activation('sigmoid'))
    model.add(layers.Conv2D(filters=120, kernel_size=(
        5, 5), strides=1, activation='tanh'))
    model.add(layers.Flatten())
    model.add(layers.Dense(84, activation='tanh'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(optimizer=SGD(learning_rate=0.01),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])  # lr = 0.01

    return model


def gen_server_model():
    model = models.Sequential()
    model.add(layers.Conv2D(filters=6, kernel_size=(5, 5),
                            strides=1, activation='sigmoid', input_shape=(32, 32, 1)))
    model.add(layers.AveragePooling2D(pool_size=2, strides=2))
    model.add(layers.Activation('tanh'))
    model.add(layers.Conv2D(filters=16, kernel_size=(
        5, 5), strides=1, activation='tanh'))
    model.add(layers.AveragePooling2D(pool_size=2, strides=2))
    model.add(layers.Activation('sigmoid'))
    model.add(layers.Conv2D(filters=120, kernel_size=(
        5, 5), strides=1, activation='tanh'))
    model.add(layers.Flatten())
    model.add(layers.Dense(84, activation='tanh'))
    model.add(layers.Dense(10, activation='softmax'))

    return model

In [3]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

x_train = x_train.reshape(60000, 28, 28, 1) / 255
x_test = x_test.reshape(10000, 28, 28, 1) / 255
x_train = np.pad(x_train, ((0, 0), (2, 2), (2, 2), (0, 0)), 'constant')
x_test = np.pad(x_test, ((0, 0), (2, 2), (2, 2), (0, 0)), 'constant')
idx = np.argsort(y_test)
x_train_sorted = x_test[idx]
y_train_sorted = y_test[idx]

UE_NUM = 20

UE = []
for _ in range(UE_NUM):
    UE.append({"x_train": [], "y_train": []})

random.seed(45)
seed = random.randint(1,1000000)
print(seed)
print(f'seed : 235511')
tf.random.set_seed(235511) # 100 좋은거, 12: 5 클러스터, 403092, 235511, 130670 : 6 클러스터, 192150 : 20

total = 0

x_eval_dataset = x_test.copy()
y_eval_dataset = y_test.copy()

for i in range(UE_NUM):
#     random_start = random.randint(1,30000)
#     end = random.randint(30001, 50000)
#     print(random_start, end)
#     UE[i]['x_train'] = x_train[random_start:end+10000]
#     UE[i]['y_train'] = y_train[random_start:end+10000]
    random_start = random.randint(1,30000)
    data_num = random.randint(5000, 30000)
    print(random_start, random_start+data_num)
    UE[i]['x_train'] = x_train[random_start:random_start+data_num]
    UE[i]['y_train'] = y_train[random_start:random_start+data_num]

x_train, x_test, y_train, y_test = [], [], [], []
for i in range(UE_NUM):
    x_train_temp, x_test_temp, y_train_temp, y_test_temp = train_test_split(
        UE[i]['x_train'], UE[i]['y_train'], test_size=0.2, random_state=45)
    x_train.append(x_train_temp)
    x_test.append(x_test_temp)
    y_train.append(y_train_temp)
    y_test.append(y_test_temp)

seed : 235511
8909 27595
15992 29435
2684 17618
11100 16807
2371 23239
362 9125
9302 18159
10193 39555
21277 27448
29463 54940
2184 11608
8900 38048
5051 16965
27790 57717
23505 42542
2457 27541
13374 23053
22953 29827
1055 11698
10508 23761


In [4]:
for i, x in enumerate(UE):
    print(f"Client {i+1}'s data amount : {len(x['x_train'])}")

Client 1's data amount : 18686
Client 2's data amount : 13443
Client 3's data amount : 14934
Client 4's data amount : 5707
Client 5's data amount : 20868
Client 6's data amount : 8763
Client 7's data amount : 8857
Client 8's data amount : 29362
Client 9's data amount : 6171
Client 10's data amount : 25477
Client 11's data amount : 9424
Client 12's data amount : 29148
Client 13's data amount : 11914
Client 14's data amount : 29927
Client 15's data amount : 19037
Client 16's data amount : 25084
Client 17's data amount : 9679
Client 18's data amount : 6874
Client 19's data amount : 10643
Client 20's data amount : 13253


In [5]:
simulation_num = 1
for num in range(simulation_num):
    wandb.init(project='Federated Learning (KCI)',
               name=f'0929 Simulation 2', entity='yhkim')
    save_path = f'0929_simulation_result/3'
    os.mkdir(save_path)
    
    cluster_result = {}

    learning_result_list = []
    for i in range(UE_NUM):
        model = gen_UE_model()

        learning_result_list.append(model.fit(
            x_train[i], y_train[i], batch_size=100, epochs=1, validation_data=(x_test[i], y_test[i])))
        tf.keras.backend.clear_session()

    UE_weights = []

    for model in learning_result_list:
        layer_weights = {}
        for x in model.model.layers:
            if len(x.get_weights()) > 0:
                layer_weights[x.name] = x.get_weights()
        UE_weights.append(layer_weights)

    server_model = gen_server_model()  # FL 서버 모델 생성

    sum_weights = {}

    for i in range(len(list(UE_weights[0].keys()))):
        weight_shape = [0]
        bias_shape = [0]
        for dim in UE_weights[0][list(UE_weights[0].keys())[i]][0].shape:
            weight_shape.append(dim)
        for dim in UE_weights[0][list(UE_weights[0].keys())[i]][1].shape:
            bias_shape.append(dim)
        sum_weights.update({list(UE_weights[0].keys())[i]: {
            'weight': np.empty(weight_shape), 'bias': np.empty(bias_shape)}})

        for UE in UE_weights:
            sum_weights[list(UE.keys())[i]]['weight'] = np.append(
                sum_weights[list(UE.keys())[i]]['weight'], [UE[list(UE.keys())[i]][0]], axis=0)
            sum_weights[list(UE.keys())[i]]['bias'] = np.append(
                sum_weights[list(UE.keys())[i]]['bias'], [UE[list(UE.keys())[i]][1]], axis=0)

    for layer in sum_weights.keys():
        for model_layer in server_model.layers:
            if layer == model_layer.name:
                model_layer.set_weights([np.mean(sum_weights[layer]['weight'], axis=0), np.mean(
                    sum_weights[layer]['bias'], axis=0)])

    server_model.compile(
        optimizer=SGD(learning_rate=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    result = server_model.evaluate(
        x=x_eval_dataset, y=y_eval_dataset, batch_size=128)
    tf.keras.backend.clear_session()
    print('Initial Round -- test loss, test acc:', result)
    split_result = split_ue_group(UE_weights, UE_NUM)
    print(split_result)

    server_model.save(f'{save_path}/fl_model_gap')

    for round in range(100):  # Communication Round, Global epoch
        learning_result_list = []
        
        for cluster_num, cluster in enumerate(split_result):
            cluster_learning_result_list = []
            cluster_list = []
            for i in cluster:
                cluster_list.append(i)
                
                i = int(i)
                if round == 0:
                    model = tf.keras.models.load_model(
                        f'{save_path}/fl_model_gap')
                    model.compile(optimizer=SGD(learning_rate=0.01),
                                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
                    cluster_learning_result_list.append(model.fit(
                        x_train[i], y_train[i], batch_size=100, epochs=1, validation_data=(x_test[i], y_test[i])))
                    tf.keras.backend.clear_session()
                else:
                    model = tf.keras.models.load_model(
                        f'{save_path}/cluster{cluster_num}')
                    model.compile(optimizer=SGD(learning_rate=0.01),
                                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
                    cluster_learning_result_list.append(model.fit(
                        x_train[i], y_train[i], batch_size=100, epochs=1, validation_data=(x_test[i], y_test[i])))
                    tf.keras.backend.clear_session()

            wandb.log({f'cluster{cluster_num}':cluster_list}, commit=False)
            
            UE_weights = []

            for model in cluster_learning_result_list:
                layer_weights = {}
                for x in model.model.layers:
                    if len(x.get_weights()) > 0:
                        layer_weights[x.name] = x.get_weights()
                UE_weights.append(layer_weights)

            server_model = gen_server_model()
            tf.keras.backend.clear_session()

            sum_weights = {}

            for i in range(len(list(UE_weights[0].keys()))):
                weight_shape = [0]
                bias_shape = [0]
                for dim in UE_weights[0][list(UE_weights[0].keys())[i]][0].shape:
                    weight_shape.append(dim)
                for dim in UE_weights[0][list(UE_weights[0].keys())[i]][1].shape:
                    bias_shape.append(dim)
                sum_weights.update({list(UE_weights[0].keys())[i]: {
                    'weight': np.empty(weight_shape), 'bias': np.empty(bias_shape)}})

                for UE in UE_weights:
                    sum_weights[list(UE.keys())[i]]['weight'] = np.append(
                        sum_weights[list(UE.keys())[i]]['weight'], [UE[list(UE.keys())[i]][0]], axis=0)
                    sum_weights[list(UE.keys())[i]]['bias'] = np.append(
                        sum_weights[list(UE.keys())[i]]['bias'], [UE[list(UE.keys())[i]][1]], axis=0)

            for layer in sum_weights.keys():
                for model_layer in server_model.layers:
                    if layer == model_layer.name:
                        model_layer.set_weights([np.mean(sum_weights[layer]['weight'], axis=0), np.mean(
                            sum_weights[layer]['bias'], axis=0)])

            server_model.compile(
                optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            result = server_model.evaluate(
                x=x_eval_dataset, y=y_eval_dataset, batch_size=128)
            print(f'Round {round} -- test loss, test acc: {result}')

            server_model.save(f'{save_path}/cluster{cluster_num}')
            cluster_result[f'cluster{cluster_num}_acc'] = result[1]
            cluster_result[f'cluster{cluster_num}_loss'] = result[0]
            
            wandb.log({f'cluster{cluster_num}_acc':result[1], f'cluster{cluster_num}_loss':result[0]}, commit=False)
        
        wandb.log({'global epoch':round+1})
    wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yhkim (use `wandb login --relogin` to force relogin)
C:\Users\user\anaconda3\envs\tensor\lib\site-packages\IPython\html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


79/79 [==============================] - 0s 2ms/step - loss: 2.3025 - accuracy: 0.1009
Initial Round -- test loss, test acc: [2.302480697631836, 0.10090000182390213]
[[18], [0, 8, 12], [1, 6, 7, 9, 10, 16], [3, 5, 14, 19], [4, 11, 13, 17], [2, 15]]
79/79 [==============================] - 0s 1ms/step - loss: 2.3020 - accuracy: 0.1135
Round 0 -- test loss, test acc: [2.302027702331543, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3020 - accuracy: 0.1135
Round 0 -- test loss, test acc: [2.3020358085632324, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3020 - accuracy: 0.1135
Round 0 -- test loss, test acc: [2.3019518852233887, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3020 - accuracy: 0.1135
Round 0 -- test loss, test acc: [2.30198335647583, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3019 - accuracy: 0.1135
Round 0 -- test loss, t

79/79 [==============================] - 0s 1ms/step - loss: 2.3016 - accuracy: 0.1135
Round 1 -- test loss, test acc: [2.301640510559082, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3015 - accuracy: 0.1135
Round 1 -- test loss, test acc: [2.3014957904815674, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3014 - accuracy: 0.1135
Round 1 -- test loss, test acc: [2.301389217376709, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3015 - accuracy: 0.1135
Round 2 -- test loss, test acc: [2.3015406131744385, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3015 - accuracy: 0.1135
Round 2 -- test loss, test acc: [2.301527500152588, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3014 - accuracy: 0.1135
Round 2 -- test loss, test acc: [2.3013932704925537, 0.11349999904632568]
79/79 [==============================

79/79 [==============================] - 0s 1ms/step - loss: 2.3012 - accuracy: 0.1135
Round 3 -- test loss, test acc: [2.3011815547943115, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3011 - accuracy: 0.1135
Round 3 -- test loss, test acc: [2.3011205196380615, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3014 - accuracy: 0.1135
Round 4 -- test loss, test acc: [2.301370620727539, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3013 - accuracy: 0.1135
Round 4 -- test loss, test acc: [2.3013064861297607, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3012 - accuracy: 0.1135
Round 4 -- test loss, test acc: [2.301168441772461, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3011 - accuracy: 0.1135
Round 4 -- test loss, test acc: [2.3011367321014404, 0.11349999904632568]
79/79 [=============================

79/79 [==============================] - 0s 1ms/step - loss: 2.3011 - accuracy: 0.1135
Round 8 -- test loss, test acc: [2.3010547161102295, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3010 - accuracy: 0.1135
Round 8 -- test loss, test acc: [2.300976276397705, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3010 - accuracy: 0.1135
Round 8 -- test loss, test acc: [2.3010473251342773, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3011 - accuracy: 0.1135
Round 8 -- test loss, test acc: [2.3011393547058105, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3014 - accuracy: 0.1135
Round 9 -- test loss, test acc: [2.3014392852783203, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3012 - accuracy: 0.1135
Round 9 -- test loss, test acc: [2.3012008666992188, 0.11349999904632568]
79/79 [============================

...

79/79 [==============================] - 0s 1ms/step - loss: 2.3012 - accuracy: 0.1135
Round 90 -- test loss, test acc: [2.301206350326538, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3018 - accuracy: 0.1135
Round 91 -- test loss, test acc: [2.301811456680298, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3013 - accuracy: 0.1135
Round 91 -- test loss, test acc: [2.301269769668579, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3011 - accuracy: 0.1135
Round 91 -- test loss, test acc: [2.301056385040283, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3010 - accuracy: 0.1135
Round 91 -- test loss, test acc: [2.300983190536499, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3010 - accuracy: 0.1135
Round 91 -- test loss, test acc: [2.3010430335998535, 0.11349999904632568]
79/79 [==========================

79/79 [==============================] - 0s 1ms/step - loss: 2.3011 - accuracy: 0.1135
Round 95 -- test loss, test acc: [2.3010566234588623, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3010 - accuracy: 0.1135
Round 95 -- test loss, test acc: [2.300983190536499, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3010 - accuracy: 0.1135
Round 95 -- test loss, test acc: [2.3010430335998535, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3012 - accuracy: 0.1135
Round 95 -- test loss, test acc: [2.301206111907959, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3018 - accuracy: 0.1135
Round 96 -- test loss, test acc: [2.301811456680298, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3013 - accuracy: 0.1135
Round 96 -- test loss, test acc: [2.301269769668579, 0.11349999904632568]
79/79 [=========================

79/79 [==============================] - 0s 1ms/step - loss: 2.3010 - accuracy: 0.1135
Round 97 -- test loss, test acc: [2.3010430335998535, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3012 - accuracy: 0.1135
Round 97 -- test loss, test acc: [2.301206111907959, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3018 - accuracy: 0.1135
Round 98 -- test loss, test acc: [2.301811456680298, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3013 - accuracy: 0.1135
Round 98 -- test loss, test acc: [2.30126953125, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3011 - accuracy: 0.1135
Round 98 -- test loss, test acc: [2.3010566234588623, 0.11349999904632568]
79/79 [==============================] - 0s 1ms/step - loss: 2.3010 - accuracy: 0.1135
Round 98 -- test loss, test acc: [2.300983190536499, 0.11349999904632568]
79/79 [=============================

cluster0_acc,0.1135
cluster0_loss,2.30181
cluster1_acc,0.1135
cluster1_loss,2.30127
cluster2_acc,0.1135
cluster2_loss,2.30106
cluster3_acc,0.1135
cluster3_loss,2.30098
cluster4_acc,0.1135
cluster4_loss,2.30104
cluster5_acc,0.1135


cluster0_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
cluster0_loss,█▃▁▁▂▃▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
cluster1_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
cluster1_loss,█▄▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
cluster2_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
cluster2_loss,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
cluster3_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
cluster3_loss,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
cluster4_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
cluster4_loss,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
cluster5_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
